In [ ]:
import arcpy
from arcpy import na 
import arcpy.nax
import os
import time

modes= ['Walk','Bike','Transit','Drive']

def calculate_travel_time(mode, origin_shp_path, travel_time_field, hptripid_field_name):
    # Create a mapping for the mode to the respective field in ODCostMatrix and SearchCursor
    mode_field_mapping = {
        'Walk': 'Mins_PedOnly',
        'Bike': 'Mins_BikeOnly',
        'Drive': 'Mins_DriveOnly',
        'Transit': 'Mins_PedTransitOnly'
    }
    mode_field = mode_field_mapping[mode]

    # Process each hptripid value in the origin shapefile
    with arcpy.da.SearchCursor(origin_shp_path, [hptripid_field_name]) as cursor:
        for row in cursor:
            hptripid = row[0]
            converted_hptripid = hptripid.replace('.', '_')
            int_shp_path = os.path.join(int_service_points_dir, f"POI_{mode}_{converted_hptripid}.shp")

            if arcpy.Exists(int_shp_path) and any(row[0] == hptripid for row in arcpy.da.SearchCursor(int_shp_path, [hptripid_field_name])):
                # Create a new OD Cost Matrix layer for the current hptripid
                out_od_cost_matrix = arcpy.na.MakeODCostMatrixLayer(network_dataset, 
                                                                    "ODCostMatrix", 
                                                                    mode_field
                                                                    )
                out_od_cost_matrix_object = out_od_cost_matrix.getOutput(0)

                sublayer_names = arcpy.na.GetNAClassNames(out_od_cost_matrix_object)
                origins_sublayer_name = sublayer_names["Origins"]
                destinations_sublayer_name = sublayer_names["Destinations"]

                # Delete rows from the Origins and Destinations sublayers to ensure they are empty
                arcpy.DeleteRows_management(origins_sublayer_name)
                arcpy.DeleteRows_management(destinations_sublayer_name)

                # Create a temporary feature layer for the specific hptripid
                temp_layer_name = "temp_origin_layer"
                arcpy.MakeFeatureLayer_management(origin_shp_path, temp_layer_name, f"{hptripid_field_name} = '{hptripid}'")

                # Set the search tolerance
                search_tolerance = "300 Meters"
                search_criteria = [["BikePedAuto", "SHAPE"]]
                
                # Add this temporary layer as the origins
                arcpy.na.AddLocations(out_od_cost_matrix_object, "Origins", temp_layer_name, field_mappings="", 
                                      snap_to_position_along_network="SNAP", snap_offset="300 Meters",
                                      search_tolerance=search_tolerance, search_criteria=search_criteria)

                # Delete the temporary layer
                arcpy.Delete_management(temp_layer_name)

                # Add the destinations
                arcpy.na.AddLocations(out_od_cost_matrix_object, "Destinations", int_shp_path,field_mappings="Name int_OBJID #",
                                      snap_to_position_along_network="SNAP", snap_offset="300 Meters",
                                    search_tolerance=search_tolerance,search_criteria=search_criteria
                                        )

                # Solve the OD Cost Matrix
                arcpy.na.Solve(out_od_cost_matrix_object, terminate_on_solve_error="CONTINUE")

                # Directly assign the name of the lines sublayer
                lines_sublayer = "Lines"

                with arcpy.da.SearchCursor(lines_sublayer, ["OriginID", "DestinationID", f"Total_{mode_field}"]) as sCursor:
                    for sRow in sCursor:
                        origin_id = sRow[0]
                        dest_id = sRow[1]
                        travel_time = sRow[2]

                        # Update the travel time field in the int_shp_path shapefile with the computed travel time
                        with arcpy.da.UpdateCursor(int_shp_path, ["int_OBJID", travel_time_field]) as uCursor:
                            for uRow in uCursor:
                                if uRow[0] == dest_id:  # Assuming that int_OBJID matches the DestinationID
                                    uRow[1] = travel_time
                                    uCursor.updateRow(uRow)

                # Optionally delete this OD matrix layer to save memory
                arcpy.Delete_management(out_od_cost_matrix_object)

                # Save the updated shapefile to the output directory
                output_shp_path = os.path.join(output_directory, f"FOS_{mode}_{converted_hptripid}.shp")
                arcpy.CopyFeatures_management(int_shp_path, output_shp_path)

            else:
                print(f"Shapefile {int_shp_path} does not exist. Skipping...")

def remove_rows_based_on_total_time(origin_shp_path, output_directory, hptripid_field_name):
    with arcpy.da.SearchCursor(origin_shp_path, [hptripid_field_name, 'Total_T']) as cursor:
        for row in cursor:
            hptripid = row[0]
            total_t = row[1]
            converted_hptripid = hptripid.replace('.', '_')
            output_shp_path = os.path.join(output_directory, f"FOS_{mode}_{converted_hptripid}.shp")
            
            # Check if the shapefile exists
            if arcpy.Exists(output_shp_path):
                # Rows to be deleted
                rows_to_delete = []
                # First, look for rows where either trav_t or trav_t_itd is 0
                with arcpy.da.SearchCursor(output_shp_path, ['OID@', 'trav_t', 'trav_t_itd']) as zero_check_cursor:
                    for zero_row in zero_check_cursor:
                        if zero_row[1] == 0 or zero_row[2] == 0:
                            rows_to_delete.append(zero_row[0])
                
                # If any rows were marked for deletion, delete them
                if rows_to_delete:
                    with arcpy.da.UpdateCursor(output_shp_path, ['OID@']) as del_cursor:
                        for del_row in del_cursor:
                            if del_row[0] in rows_to_delete:
                                del_cursor.deleteRow()

                # Reset the rows_to_delete list for the next check
                rows_to_delete = []
                # Calculate the sum of trav_t and trav_t_itd for each row and compare
                with arcpy.da.SearchCursor(output_shp_path, ['OID@', 'trav_t', 'trav_t_itd']) as travel_cursor:
                    for travel_row in travel_cursor:
                        oid = travel_row[0]
                        sum_travel_time = (travel_row[1] or 0) + (travel_row[2] or 0)
                        if total_t < sum_travel_time:
                            rows_to_delete.append(oid)
                # Delete rows that don't meet the criteria
                if rows_to_delete:
                    with arcpy.da.UpdateCursor(output_shp_path, ['OID@']) as del_cursor:
                        for del_row in del_cursor:
                            if del_row[0] in rows_to_delete:
                                del_cursor.deleteRow()

                # Print the Total_T value and remaining counts
                remaining_count = arcpy.GetCount_management(output_shp_path)
                print(f"For hptripid {hptripid} with Total_T value of {total_t}:")
                print(f"Shapefile {output_shp_path} has {remaining_count} rows remaining.")
                print("-"*50)  # separator for clarity

def add_and_populate_fields(mode, int_service_points_dir):
    # List all the shapefiles in the directory
    shapefiles = [f for f in os.listdir(int_service_points_dir) if f.endswith('.shp')]
    
    # Loop through each shapefile
    for shp in shapefiles:
        # Check if the shapefile name matches the format "POI_{mode}_{hptripid}.shp"
        if shp.startswith(f"POI_{mode}_"):
            # Construct the full path to the shapefile
            shp_path = os.path.join(int_service_points_dir, shp)
            
            # Add a new field named "int_OBJECTID" with data type "LONG"
            arcpy.AddField_management(shp_path, "int_OBJID", "LONG")
            arcpy.AddField_management(shp_path, "trav_t", "DOUBLE")
            arcpy.AddField_management(shp_path, "trav_t_itd", "DOUBLE")

            # Populate the new field with values starting from 1
            with arcpy.da.UpdateCursor(shp_path, "int_OBJID") as cursor:
                count = 1
                for row in cursor:
                    row[0] = count
                    cursor.updateRow(row)
                    count += 1

# Record the start time
start_time = time.time()  

# Set environment
workspace = rf"E:\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\MyPwoDSTPProject.gdb"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True 

network_dataset = fr"E:\Space-Time-Prism-Model\PwoD STP\MM_NetworkDataset_03092022.gdb\NetworkDataset\NetworkDataset_ND"

for mode in modes:
    origin_shp_path = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles\origin_{mode}_filtered.shp"
    destn_shp_path = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles\destn_{mode}_filtered.shp"
    int_service_points_dir = rf"E:\Space-Time-Prism-Model\PwoD STP\POI_after_Ser_hr\{mode}_POI_after_ser_hr\Reprojected"
    output_directory = rf"E:\Space-Time-Prism-Model\PwoD STP\FOS_Count\{mode}_FOS"
    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)

    # Call the function to add and populate fields for each mode-specific shapefile
    add_and_populate_fields(mode, int_service_points_dir)  

    hptripid_field_origin = [field.name for field in arcpy.ListFields(origin_shp_path) if field.name.lower() == 'hptripid'][0]
    hptripid_field_destn = [field.name for field in arcpy.ListFields(destn_shp_path) if field.name.lower() == 'hptripid'][0]

    # Calculate travel time for origin shapefile
    calculate_travel_time(mode, origin_shp_path, "trav_t", hptripid_field_origin)
    # Calculate travel time for destination shapefile
    calculate_travel_time(mode, destn_shp_path, "trav_t_itd", hptripid_field_destn)
    # Call the function to remove rows based on the given criteria
    remove_rows_based_on_total_time(origin_shp_path, output_directory, hptripid_field_origin)

    # Record the end time
    end_time = time.time()
    # Calculate the total processing time
    processing_time = end_time - start_time
    print(f"The script took {processing_time} seconds to complete.")